In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [2]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Dropout

In [3]:
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [4]:
df1 = pd.read_csv('./MovementAAL/dataset/MovementAAL_RSS_1.csv')
df2 = pd.read_csv('./MovementAAL/dataset/MovementAAL_RSS_2.csv')

In [5]:
df1.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.90476,-0.48,0.28571,0.30
1,-0.57143,-0.32,0.14286,0.30
2,-0.38095,-0.28,-0.14286,0.35
3,-0.28571,-0.20,-0.47619,0.35
4,-0.14286,-0.20,0.14286,-0.20


In [6]:
df2.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.57143,-0.20,0.71429,0.50
1,-0.76190,-0.48,0.76190,-0.25
2,-0.85714,-0.60,0.85714,0.55
3,-0.76190,-0.40,0.71429,0.60
4,-0.76190,-0.84,0.85714,0.45


In [7]:
df1.shape

(27, 4)

In [8]:
df2.shape

(26, 4)

In [9]:
path = './MovementAAL/dataset/MovementAAL_RSS_'
sequences = list()
for i in range(1,315):
    file_path = path + str(i) + '.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)

./MovementAAL/dataset/MovementAAL_RSS_1.csv
./MovementAAL/dataset/MovementAAL_RSS_2.csv
./MovementAAL/dataset/MovementAAL_RSS_3.csv
./MovementAAL/dataset/MovementAAL_RSS_4.csv
./MovementAAL/dataset/MovementAAL_RSS_5.csv
./MovementAAL/dataset/MovementAAL_RSS_6.csv
./MovementAAL/dataset/MovementAAL_RSS_7.csv
./MovementAAL/dataset/MovementAAL_RSS_8.csv
./MovementAAL/dataset/MovementAAL_RSS_9.csv
./MovementAAL/dataset/MovementAAL_RSS_10.csv
./MovementAAL/dataset/MovementAAL_RSS_11.csv
./MovementAAL/dataset/MovementAAL_RSS_12.csv
./MovementAAL/dataset/MovementAAL_RSS_13.csv
./MovementAAL/dataset/MovementAAL_RSS_14.csv
./MovementAAL/dataset/MovementAAL_RSS_15.csv
./MovementAAL/dataset/MovementAAL_RSS_16.csv
./MovementAAL/dataset/MovementAAL_RSS_17.csv
./MovementAAL/dataset/MovementAAL_RSS_18.csv
./MovementAAL/dataset/MovementAAL_RSS_19.csv
./MovementAAL/dataset/MovementAAL_RSS_20.csv
./MovementAAL/dataset/MovementAAL_RSS_21.csv
./MovementAAL/dataset/MovementAAL_RSS_22.csv
./MovementAAL/datas

./MovementAAL/dataset/MovementAAL_RSS_187.csv
./MovementAAL/dataset/MovementAAL_RSS_188.csv
./MovementAAL/dataset/MovementAAL_RSS_189.csv
./MovementAAL/dataset/MovementAAL_RSS_190.csv
./MovementAAL/dataset/MovementAAL_RSS_191.csv
./MovementAAL/dataset/MovementAAL_RSS_192.csv
./MovementAAL/dataset/MovementAAL_RSS_193.csv
./MovementAAL/dataset/MovementAAL_RSS_194.csv
./MovementAAL/dataset/MovementAAL_RSS_195.csv
./MovementAAL/dataset/MovementAAL_RSS_196.csv
./MovementAAL/dataset/MovementAAL_RSS_197.csv
./MovementAAL/dataset/MovementAAL_RSS_198.csv
./MovementAAL/dataset/MovementAAL_RSS_199.csv
./MovementAAL/dataset/MovementAAL_RSS_200.csv
./MovementAAL/dataset/MovementAAL_RSS_201.csv
./MovementAAL/dataset/MovementAAL_RSS_202.csv
./MovementAAL/dataset/MovementAAL_RSS_203.csv
./MovementAAL/dataset/MovementAAL_RSS_204.csv
./MovementAAL/dataset/MovementAAL_RSS_205.csv
./MovementAAL/dataset/MovementAAL_RSS_206.csv
./MovementAAL/dataset/MovementAAL_RSS_207.csv
./MovementAAL/dataset/MovementAAL_

In [10]:
len(sequences)

314

In [11]:
targets = pd.read_csv('./MovementAAL/dataset/MovementAAL_target.csv')
targets = targets.values[:,1]
targets

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [12]:
#sequences[313]

In [13]:
groups = pd.read_csv('./MovementAAL/groups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]

In [14]:
#groups

### PRE-PROCESSING

In [15]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()

count    314.000000
mean      42.028662
std       16.185303
min       19.000000
25%       26.000000
50%       41.000000
75%       56.000000
max      129.000000
dtype: float64

In [16]:
#Padding the sequence with the values in last row to max length
to_pad = 129
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 60
from keras.preprocessing import sequence
seq_len = 60
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [17]:
final_seq.shape

(314, 60, 4)

In [18]:
#final_seq[0]

In [19]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

In [44]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
train_target = (train_target+1)/2
train_target=np.reshape(train_target,(-1,1))

validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2
validation_target=np.reshape(validation_target,(-1,1))

test_target = np.array(test_target)
test_target = (test_target+1)/2
test_target=np.reshape(test_target,(-1,1))

In [45]:
train.shape, validation.shape, test.shape, train_target.shape

((106, 60, 4), (104, 60, 4), (104, 60, 4), (106, 1))

In [46]:
seq_len

60

# TRY1: Building time series classification model

In [60]:
# mcfly
import mcfly, os
np.random.seed(2)

In [61]:
num_classes = train_target.shape[1]
metric = 'accuracy'
models = mcfly.modelgen.generate_models(train.shape,
                                        number_of_classes=num_classes,
                                        number_of_models = 8,
                                        metrics=[metric])

Set maximum kernel size for InceptionTime models to number of timesteps.
Set maximum kernel size for InceptionTime models to number of timesteps.


### Inspect models

In [62]:
models_to_print = range(len(models))
for i, item in enumerate(models):
    if i in models_to_print:
        model, params, model_types = item
        print("-------------------------------------------------------------------------------------------------------")
        print("Model " + str(i))
        print(" ")
        print("Hyperparameters:")
        print(params)
        print(" ")
        print("Model description:")
        model.summary()
        print(" ")
        print("Model type:")
        print(model_types)
        print(" ")

-------------------------------------------------------------------------------------------------------
Model 0
 
Hyperparameters:
{'learning_rate': 0.005481464347736465, 'regularization_rate': 0.010209289921137055, 'filters': array([ 57,  73,  41, 100,  30,  47]), 'fc_hidden_nodes': 689}
 
Model description:
Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_602 (Bat (None, 60, 4)             16        
_________________________________________________________________
conv1d_849 (Conv1D)          (None, 60, 57)            741       
_________________________________________________________________
batch_normalization_603 (Bat (None, 60, 57)            228       
_________________________________________________________________
activation_367 (Activation)  (None, 60, 57)            0         
_________________________________________________________________
conv1d

In [63]:
models

[(<tensorflow.python.keras.engine.sequential.Sequential at 0x1c6709f8b20>,
  {'learning_rate': 0.005481464347736465,
   'regularization_rate': 0.010209289921137055,
   'filters': array([ 57,  73,  41, 100,  30,  47]),
   'fc_hidden_nodes': 689},
  'CNN'),
 (<tensorflow.python.keras.engine.training.Model at 0x1c75583c280>,
  {'learning_rate': 0.0394693672382836,
   'regularization_rate': 0.002879158913712458,
   'network_depth': 5,
   'min_filters_number': 83,
   'max_kernel_size': 15},
  'ResNet'),
 (<tensorflow.python.keras.engine.training.Model at 0x1c7560a3b20>,
  {'learning_rate': 0.016768945015015333,
   'regularization_rate': 0.0022981482466674346,
   'network_depth': 3,
   'filters_number': 78,
   'max_kernel_size': 16},
  'InceptionTime'),
 (<tensorflow.python.keras.engine.sequential.Sequential at 0x1c671650940>,
  {'learning_rate': 0.013689688066052806,
   'regularization_rate': 0.04481465394760804,
   'filters': [90, 62, 86],
   'lstm_dims': [14, 100, 73]},
  'DeepConvLSTM'),

### Compare models

In [64]:
# Define directory where the results, e.g. json file, will be stored
resultpath = os.path.join('.', 'models')
if not os.path.exists(resultpath):
    os.makedirs(resultpath)

### Train models

In [65]:
from mcfly.find_architecture import train_models_on_samples

outputfile = os.path.join(resultpath, 'modelcomparison.json')
histories, val_accuracies, val_losses = train_models_on_samples(train, train_target,
                                                                validation, validation_target,
                                                                models, nr_epochs=200,
                                                                subset_size=300,
                                                                early_stopping_patience = 5,
                                                                verbose=True,
                                                                outputfile=outputfile,
                                                                metric=metric)
print('Details of the training process were stored in ',outputfile)

Generated models will be trained on subset of the data (subset size: 300).
Training model 0 CNN
Epoch 1/200


c:\python38\lib\site-packages\mcfly\find_architecture.py:134: UserWarning: Argument 'metric' is deprecated and will be ignored.
  warnings.warn("Argument 'metric' is deprecated and will be ignored.")


6/6 [==============================] - 0s 78ms/step - loss: 6.7689 - accuracy: 0.4906 - val_loss: 4.7027 - val_accuracy: 0.5000
Epoch 2/200
6/6 [==============================] - 0s 36ms/step - loss: 4.4956 - accuracy: 0.4906 - val_loss: 3.8453 - val_accuracy: 0.5000
Epoch 3/200
6/6 [==============================] - 0s 36ms/step - loss: 3.2824 - accuracy: 0.4906 - val_loss: 2.3842 - val_accuracy: 0.5000
Epoch 4/200
6/6 [==============================] - 0s 34ms/step - loss: 2.0466 - accuracy: 0.4906 - val_loss: 1.5786 - val_accuracy: 0.5000
Epoch 5/200
6/6 [==============================] - 0s 33ms/step - loss: 1.3768 - accuracy: 0.4906 - val_loss: 1.0523 - val_accuracy: 0.5000
Epoch 6/200
6/6 [==============================] - 0s 34ms/step - loss: 0.9135 - accuracy: 0.4906 - val_loss: 0.7289 - val_accuracy: 0.5000
Epoch 7/200
6/6 [==============================] - 0s 34ms/step - loss: 0.6538 - accuracy: 0.4906 - val_loss: 0.5245 - val_accuracy: 0.5000
Epoch 8/200
6/6 [===============

6/6 [==============================] - 1s 106ms/step - loss: 2.5269e-04 - accuracy: 0.4906 - val_loss: 1.5350e-04 - val_accuracy: 0.5000
Epoch 23/200
6/6 [==============================] - 1s 109ms/step - loss: 1.1827e-04 - accuracy: 0.4906 - val_loss: 6.9880e-05 - val_accuracy: 0.5000
Epoch 24/200
6/6 [==============================] - 1s 112ms/step - loss: 5.3090e-05 - accuracy: 0.4906 - val_loss: 3.0574e-05 - val_accuracy: 0.5000
Epoch 25/200
6/6 [==============================] - 1s 107ms/step - loss: 2.3054e-05 - accuracy: 0.4906 - val_loss: 1.3047e-05 - val_accuracy: 0.5000
Epoch 26/200
6/6 [==============================] - 1s 106ms/step - loss: 9.7985e-06 - accuracy: 0.4906 - val_loss: 5.5878e-06 - val_accuracy: 0.5000
Epoch 27/200
6/6 [==============================] - 1s 106ms/step - loss: 4.2691e-06 - accuracy: 0.4906 - val_loss: 2.5657e-06 - val_accuracy: 0.5000
Epoch 28/200
6/6 [==============================] - 1s 106ms/step - loss: 2.0353e-06 - accuracy: 0.4906 - val_los

6/6 [==============================] - 0s 72ms/step - loss: 1.7713 - accuracy: 0.4906 - val_loss: 1.9836 - val_accuracy: 0.5000
Epoch 13/200
6/6 [==============================] - 0s 75ms/step - loss: 1.7166 - accuracy: 0.4906 - val_loss: 1.4877 - val_accuracy: 0.5000
Epoch 14/200
6/6 [==============================] - 0s 73ms/step - loss: 1.7819 - accuracy: 0.4906 - val_loss: 1.8650 - val_accuracy: 0.5000
Epoch 00014: early stopping
Training model 5 ResNet
Epoch 1/200
6/6 [==============================] - 1s 122ms/step - loss: 0.0000e+00 - accuracy: 0.4906 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/200
6/6 [==============================] - 0s 64ms/step - loss: 0.0000e+00 - accuracy: 0.4906 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/200
6/6 [==============================] - 0s 73ms/step - loss: 0.0000e+00 - accuracy: 0.4906 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 4/200
6/6 [==============================] - 0s 74ms/step - loss: 0.0000e+00 - accuracy

6/6 [==============================] - 1s 126ms/step - loss: 5.7529e-04 - accuracy: 0.4906 - val_loss: 5.1433e-04 - val_accuracy: 0.5000
Epoch 49/200
6/6 [==============================] - 1s 125ms/step - loss: 4.8327e-04 - accuracy: 0.4906 - val_loss: 4.3156e-04 - val_accuracy: 0.5000
Epoch 50/200
6/6 [==============================] - 1s 125ms/step - loss: 4.0525e-04 - accuracy: 0.4906 - val_loss: 3.6147e-04 - val_accuracy: 0.5000
Epoch 51/200
6/6 [==============================] - 1s 125ms/step - loss: 3.3921e-04 - accuracy: 0.4906 - val_loss: 3.0221e-04 - val_accuracy: 0.5000
Epoch 52/200
6/6 [==============================] - 1s 125ms/step - loss: 2.8343e-04 - accuracy: 0.4906 - val_loss: 2.5222e-04 - val_accuracy: 0.5000
Epoch 53/200
6/6 [==============================] - 1s 125ms/step - loss: 2.3640e-04 - accuracy: 0.4906 - val_loss: 2.1013e-04 - val_accuracy: 0.5000
Epoch 54/200
6/6 [==============================] - 1s 124ms/step - loss: 1.9682e-04 - accuracy: 0.4906 - val_los

Epoch 103/200
6/6 [==============================] - 1s 125ms/step - loss: 9.2822e-08 - accuracy: 0.4906 - val_loss: 9.2045e-08 - val_accuracy: 0.5000
Epoch 104/200
6/6 [==============================] - 1s 125ms/step - loss: 9.2944e-08 - accuracy: 0.4906 - val_loss: 1.0578e-07 - val_accuracy: 0.5000
Epoch 105/200
6/6 [==============================] - 1s 126ms/step - loss: 9.9877e-08 - accuracy: 0.4906 - val_loss: 9.3663e-08 - val_accuracy: 0.5000
Epoch 106/200
6/6 [==============================] - 1s 125ms/step - loss: 9.4507e-08 - accuracy: 0.4906 - val_loss: 1.0106e-07 - val_accuracy: 0.5000
Epoch 107/200
6/6 [==============================] - 1s 126ms/step - loss: 9.6275e-08 - accuracy: 0.4906 - val_loss: 9.7450e-08 - val_accuracy: 0.5000
Epoch 108/200
6/6 [==============================] - 1s 126ms/step - loss: 9.6756e-08 - accuracy: 0.4906 - val_loss: 9.3789e-08 - val_accuracy: 0.5000
Epoch 00108: early stopping
Training model 7 InceptionTime
Epoch 1/200
6/6 [==================

### Inspect model performance (Visualization)

In [66]:
metric = 'accuracy'
modelcomparisons = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                                'model-type':[str(model_types) for model, params, model_types in models],
                                'train_{}'.format(metric): [history.history[metric][-1] for history in histories],
                                'train_loss': [history.history['loss'][-1] for history in histories],
                                'val_{}'.format(metric): [history.history['val_{}'.format(metric)][-1] for history in histories],
                                'val_loss': [history.history['val_loss'][-1] for history in histories]
                                })
modelcomparisons.to_csv(os.path.join(resultpath, 'modelcomparisons.csv'))

modelcomparisons

,model,model-type,train_accuracy,train_loss,val_accuracy,val_loss
0,"{'learning_rate': 0.005481464347736465, 'regul...",CNN,0.490566,1.707531e-01,0.5,1.612915e-01
1,"{'learning_rate': 0.0394693672382836, 'regular...",ResNet,0.490566,0.000000e+00,0.5,0.000000e+00
2,"{'learning_rate': 0.016768945015015333, 'regul...",InceptionTime,0.490566,0.000000e+00,0.5,0.000000e+00
3,"{'learning_rate': 0.013689688066052806, 'regul...",DeepConvLSTM,0.490566,8.884466e-08,0.5,8.940697e-08
4,"{'learning_rate': 0.02183138953555479, 'regula...",CNN,0.490566,1.781881e+00,0.5,1.865005e+00
5,"{'learning_rate': 0.022294114160952016, 'regul...",ResNet,0.490566,0.000000e+00,0.5,0.000000e+00
6,"{'learning_rate': 0.005216294589208116, 'regul...",DeepConvLSTM,0.490566,9.675642e-08,0.5,9.378899e-08
7,"{'learning_rate': 0.00010433835028846764, 'reg...",InceptionTime,0.490566,0.000000e+00,0.5,0.000000e+00


### 1) Continue iteratively: generate more models of desired type

In [67]:
num_classes = train_target.shape[1]
metric = 'accuracy'
models = mcfly.modelgen.generate_models(train.shape,
                                        number_of_classes=num_classes,
                                        number_of_models = 8,
                                        model_types = ['CNN', 'InceptionTime'],
                                        low_lr=2,
                                        metrics=[metric])

The value of low_lr is set from 1 (default) to 2
Set maximum kernel size for InceptionTime models to number of timesteps.
Set maximum kernel size for InceptionTime models to number of timesteps.
Set maximum kernel size for InceptionTime models to number of timesteps.
Set maximum kernel size for InceptionTime models to number of timesteps.


In [69]:
outputfile = os.path.join(resultpath, 'modelcomparison_2.json')
histories, val_accuracies, val_losses = train_models_on_samples(train, train_target,
                                                                validation, validation_target,
                                                                models, nr_epochs=200,
                                                                subset_size=300,
                                                                early_stopping_patience = 5,
                                                                verbose=True,
                                                                outputfile=outputfile,
                                                                metric=metric)

Generated models will be trained on subset of the data (subset size: 300).
Training model 0 CNN
Epoch 1/200


c:\python38\lib\site-packages\mcfly\find_architecture.py:134: UserWarning: Argument 'metric' is deprecated and will be ignored.
  warnings.warn("Argument 'metric' is deprecated and will be ignored.")


6/6 [==============================] - 0s 43ms/step - loss: 1.4560 - accuracy: 0.4906 - val_loss: 1.4105 - val_accuracy: 0.5000
Epoch 2/200
6/6 [==============================] - 0s 17ms/step - loss: 1.3887 - accuracy: 0.4906 - val_loss: 1.3477 - val_accuracy: 0.5000
Epoch 3/200
6/6 [==============================] - 0s 19ms/step - loss: 1.3227 - accuracy: 0.4906 - val_loss: 1.2778 - val_accuracy: 0.5000
Epoch 4/200
6/6 [==============================] - 0s 18ms/step - loss: 1.2517 - accuracy: 0.4906 - val_loss: 1.2057 - val_accuracy: 0.5000
Epoch 5/200
6/6 [==============================] - 0s 18ms/step - loss: 1.1797 - accuracy: 0.4906 - val_loss: 1.1342 - val_accuracy: 0.5000
Epoch 6/200
6/6 [==============================] - 0s 19ms/step - loss: 1.1089 - accuracy: 0.4906 - val_loss: 1.0650 - val_accuracy: 0.5000
Epoch 7/200
6/6 [==============================] - 0s 21ms/step - loss: 1.0408 - accuracy: 0.4906 - val_loss: 0.9989 - val_accuracy: 0.5000
Epoch 8/200
6/6 [===============

6/6 [==============================] - 0s 16ms/step - loss: 0.1059 - accuracy: 0.4906 - val_loss: 0.1040 - val_accuracy: 0.5000
Epoch 60/200
6/6 [==============================] - 0s 17ms/step - loss: 0.1030 - accuracy: 0.4906 - val_loss: 0.1013 - val_accuracy: 0.5000
Epoch 61/200
6/6 [==============================] - 0s 16ms/step - loss: 0.1003 - accuracy: 0.4906 - val_loss: 0.0986 - val_accuracy: 0.5000
Epoch 62/200
6/6 [==============================] - 0s 17ms/step - loss: 0.0977 - accuracy: 0.4906 - val_loss: 0.0961 - val_accuracy: 0.5000
Epoch 63/200
6/6 [==============================] - 0s 16ms/step - loss: 0.0953 - accuracy: 0.4906 - val_loss: 0.0938 - val_accuracy: 0.5000
Epoch 64/200
6/6 [==============================] - 0s 16ms/step - loss: 0.0930 - accuracy: 0.4906 - val_loss: 0.0918 - val_accuracy: 0.5000
Epoch 65/200
6/6 [==============================] - 0s 16ms/step - loss: 0.0912 - accuracy: 0.4906 - val_loss: 0.0902 - val_accuracy: 0.5000
Epoch 66/200
6/6 [========

6/6 [==============================] - 0s 20ms/step - loss: 0.0353 - accuracy: 0.4906 - val_loss: 0.0348 - val_accuracy: 0.5000
Epoch 118/200
6/6 [==============================] - 0s 20ms/step - loss: 0.0346 - accuracy: 0.4906 - val_loss: 0.0342 - val_accuracy: 0.5000
Epoch 119/200
6/6 [==============================] - 0s 20ms/step - loss: 0.0339 - accuracy: 0.4906 - val_loss: 0.0336 - val_accuracy: 0.5000
Epoch 120/200
6/6 [==============================] - 0s 19ms/step - loss: 0.0334 - accuracy: 0.4906 - val_loss: 0.0330 - val_accuracy: 0.5000
Epoch 121/200
6/6 [==============================] - 0s 18ms/step - loss: 0.0329 - accuracy: 0.4906 - val_loss: 0.0326 - val_accuracy: 0.5000
Epoch 122/200
6/6 [==============================] - 0s 21ms/step - loss: 0.0325 - accuracy: 0.4906 - val_loss: 0.0323 - val_accuracy: 0.5000
Epoch 123/200
6/6 [==============================] - 0s 20ms/step - loss: 0.0322 - accuracy: 0.4906 - val_loss: 0.0320 - val_accuracy: 0.5000
Epoch 124/200
6/6 [=

...

Epoch 55/200
6/6 [==============================] - 0s 23ms/step - loss: 0.3225 - accuracy: 0.4906 - val_loss: 0.3206 - val_accuracy: 0.5000
Epoch 56/200
6/6 [==============================] - 0s 23ms/step - loss: 0.3198 - accuracy: 0.4906 - val_loss: 0.3183 - val_accuracy: 0.5000
Epoch 57/200
6/6 [==============================] - 0s 23ms/step - loss: 0.3180 - accuracy: 0.4906 - val_loss: 0.3172 - val_accuracy: 0.5000
Epoch 58/200
6/6 [==============================] - 0s 22ms/step - loss: 0.3163 - accuracy: 0.4906 - val_loss: 0.3146 - val_accuracy: 0.5000
Epoch 59/200
6/6 [==============================] - 0s 22ms/step - loss: 0.3134 - accuracy: 0.4906 - val_loss: 0.3108 - val_accuracy: 0.5000
Epoch 60/200
6/6 [==============================] - 0s 25ms/step - loss: 0.3090 - accuracy: 0.4906 - val_loss: 0.3057 - val_accuracy: 0.5000
Epoch 61/200
6/6 [==============================] - 0s 23ms/step - loss: 0.3037 - accuracy: 0.4906 - val_loss: 0.3001 - val_accuracy: 0.5000
Epoch 62/200


6/6 [==============================] - 0s 24ms/step - loss: 0.1591 - accuracy: 0.4906 - val_loss: 0.1568 - val_accuracy: 0.5000
Epoch 00113: early stopping
Training model 3 InceptionTime
Epoch 1/200
6/6 [==============================] - 1s 196ms/step - loss: 0.0000e+00 - accuracy: 0.4906 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/200
6/6 [==============================] - 1s 126ms/step - loss: 0.0000e+00 - accuracy: 0.4906 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/200
6/6 [==============================] - 1s 136ms/step - loss: 0.0000e+00 - accuracy: 0.4906 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 4/200
6/6 [==============================] - 1s 126ms/step - loss: 0.0000e+00 - accuracy: 0.4906 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 5/200
6/6 [==============================] - 1s 122ms/step - loss: 0.0000e+00 - accuracy: 0.4906 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 6/200
6/6 [==============================] - 1s 115ms/step - l

6/6 [==============================] - 0s 30ms/step - loss: 0.0990 - accuracy: 0.4906 - val_loss: 0.0981 - val_accuracy: 0.5000
Epoch 45/200
6/6 [==============================] - 0s 28ms/step - loss: 0.0976 - accuracy: 0.4906 - val_loss: 0.0967 - val_accuracy: 0.5000
Epoch 46/200
6/6 [==============================] - 0s 28ms/step - loss: 0.0961 - accuracy: 0.4906 - val_loss: 0.0952 - val_accuracy: 0.5000
Epoch 47/200
6/6 [==============================] - 0s 35ms/step - loss: 0.0947 - accuracy: 0.4906 - val_loss: 0.0938 - val_accuracy: 0.5000
Epoch 48/200
6/6 [==============================] - 0s 30ms/step - loss: 0.0933 - accuracy: 0.4906 - val_loss: 0.0925 - val_accuracy: 0.5000
Epoch 49/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0920 - accuracy: 0.4906 - val_loss: 0.0911 - val_accuracy: 0.5000
Epoch 50/200
6/6 [==============================] - 0s 32ms/step - loss: 0.0906 - accuracy: 0.4906 - val_loss: 0.0898 - val_accuracy: 0.5000
Epoch 51/200
6/6 [========

6/6 [==============================] - 0s 27ms/step - loss: 0.0449 - accuracy: 0.4906 - val_loss: 0.0445 - val_accuracy: 0.5000
Epoch 103/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0443 - accuracy: 0.4906 - val_loss: 0.0439 - val_accuracy: 0.5000
Epoch 104/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0437 - accuracy: 0.4906 - val_loss: 0.0433 - val_accuracy: 0.5000
Epoch 105/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0431 - accuracy: 0.4906 - val_loss: 0.0428 - val_accuracy: 0.5000
Epoch 106/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0425 - accuracy: 0.4906 - val_loss: 0.0422 - val_accuracy: 0.5000
Epoch 107/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0420 - accuracy: 0.4906 - val_loss: 0.0416 - val_accuracy: 0.5000
Epoch 108/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0414 - accuracy: 0.4906 - val_loss: 0.0411 - val_accuracy: 0.5000
Epoch 109/200
6/6 [=

Epoch 160/200
6/6 [==============================] - 0s 26ms/step - loss: 0.0227 - accuracy: 0.4906 - val_loss: 0.0225 - val_accuracy: 0.5000
Epoch 161/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0224 - accuracy: 0.4906 - val_loss: 0.0222 - val_accuracy: 0.5000
Epoch 162/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0221 - accuracy: 0.4906 - val_loss: 0.0220 - val_accuracy: 0.5000
Epoch 163/200
6/6 [==============================] - 0s 28ms/step - loss: 0.0219 - accuracy: 0.4906 - val_loss: 0.0217 - val_accuracy: 0.5000
Epoch 164/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0216 - accuracy: 0.4906 - val_loss: 0.0214 - val_accuracy: 0.5000
Epoch 165/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0214 - accuracy: 0.4906 - val_loss: 0.0212 - val_accuracy: 0.5000
Epoch 166/200
6/6 [==============================] - 0s 27ms/step - loss: 0.0211 - accuracy: 0.4906 - val_loss: 0.0209 - val_accuracy: 0.5000
Epoch 

In [70]:
metric = 'accuracy'
modelcomparisons_2 = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                                'model-type':[str(model_types) for model, params, model_types in models],
                                'train_{}'.format(metric): [history.history[metric][-1] for history in histories],
                                'train_loss': [history.history['loss'][-1] for history in histories],
                                'val_{}'.format(metric): [history.history['val_{}'.format(metric)][-1] for history in histories],
                                'val_loss': [history.history['val_loss'][-1] for history in histories]
                                })
modelcomparisons_2.to_csv(os.path.join(resultpath, 'modelcomparisons_2.csv'))

modelcomparisons_2

,model,model-type,train_accuracy,train_loss,val_accuracy,val_loss
0,"{'learning_rate': 0.00037806416493937304, 'reg...",CNN,0.490566,0.017885,0.5,0.017807
1,"{'learning_rate': 0.0007123853990530968, 'regu...",InceptionTime,0.490566,0.000000,0.5,0.000000
2,"{'learning_rate': 0.00034914273335304115, 'reg...",CNN,0.490566,0.159114,0.5,0.156823
3,"{'learning_rate': 0.008889925393993863, 'regul...",InceptionTime,0.490566,0.000000,0.5,0.000000
4,"{'learning_rate': 0.00022246617439914524, 'reg...",InceptionTime,0.490566,0.000000,0.5,0.000000
5,"{'learning_rate': 0.0005081261465795578, 'regu...",CNN,0.490566,0.014766,0.5,0.014659
6,"{'learning_rate': 0.0045608494108193166, 'regu...",CNN,0.490566,0.338133,0.5,0.329399
7,"{'learning_rate': 0.0014378820648315474, 'regu...",InceptionTime,0.490566,0.000000,0.5,0.000000
